In [1]:
import os
import pandas as pd

os.chdir("./rsna-2022-cervical-spine-fracture-detection/visualize")

In [2]:
import CSFD.data.three_dimensions
import CSFD.monai.from_checkpoint
import pathlib

In [3]:
model_path = pathlib.Path("../monai/models/EfficientNetBN_folds4_test-v4.3")
cfg, ckpt_paths = CSFD.monai.from_checkpoint.load_cfg_and_checkpoints(model_path)
cfg

{'dataset': {'type': 'train', 'data_root_path': '../data/rsna-2022-cervical-spine-fracture-detection', 'train_3d_images': '../data/3d_train_images_v2', 'data_type': 'f4', 'enable_depth_resized_with_cv2': True, 'depth_range': [0.1, 0.9], 'height_range': None, 'width_range': None, 'image_2d_shape': [256, 256], 'depth': 128, 'save_images_with_specific_depth': False, 'use_normalized_batches': True, 'type_to_load': 'npz', 'target_columns': ['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'], 'cv': {'type': 'KFold', 'seed': 42, 'n_folds': 4, 'fold': None}, 'num_workers': None, 'train_batch_size': 4, 'valid_batch_size': 4, 'test_batch_size': 2}, 'model': {'seed': 42, 'name': 'EfficientNetBN', 'kwargs': {'model_name': 'efficientnet-b4', 'in_channels': 1, 'pretrained': True}, 'use_multi_sample_dropout': False, 'optimizer': {'name': 'AdamW', 'scheduler': {'name': None, 'kwargs': {}}}}, 'train': {'accelerator': 'gpu', 'devices': 1, 'precision': 16, 'seed': 42, 'max_epochs': 5, 'learning

In [4]:
cfg.dataset.cv.type = "StratifiedKFold"
df = CSFD.data.three_dimensions.get_df(cfg.dataset)
df

creating 3d images: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2018/2018 [00:00<00:00, 8462.97it/s]


,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,fold,np_images_path
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,1,../data/3d_train_images_v2/256_256/normal/norm...
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,1,../data/3d_train_images_v2/256_256/normal/norm...
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,1,../data/3d_train_images_v2/256_256/normal/norm...
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,3,../data/3d_train_images_v2/256_256/normal/norm...
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,3,../data/3d_train_images_v2/256_256/normal/norm...
...,...,...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1,3,../data/3d_train_images_v2/256_256/normal/norm...
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1,1,../data/3d_train_images_v2/256_256/normal/norm...
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0,0,../data/3d_train_images_v2/256_256/normal/norm...
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0,3,../data/3d_train_images_v2/256_256/normal/norm...


In [5]:
import plotly.express as px

In [ ]:
fig = px.histogram(df, x="patient_overall", color="fold", barmode="group")
fig.show()

In [30]:
import numpy as np
import tqdm
images = np.load(df["np_images_path"].iloc[0])["arr_0"]
# images = CSFD.data.three_dimensions.resize_depth(images, depth=cfg.dataset.depth, depth_range=cfg.dataset.depth_range, enable_depth_resized_with_cv2=cfg.dataset.enable_depth_resized_with_cv2)
np.histogram(images.flatten(), bins=np.linspace(0, 255, 300))

images.shape, images.size

((128, 256, 256), 8388608)

In [ ]:
fig = px.imshow(images[20:30],
                animation_frame=0,
                color_continuous_scale='gray',
                zmin=0, zmax=255
                )
fig.show()